# Image classification

First, we build an image classification pipeline to differentiate between vehicle and non-vehicle images.

## Load images

In [1]:
import glob
import matplotlib.image as mpimg

def load_images(pattern):
    pathnames = glob.glob(pattern)
    return [mpimg.imread(x) for x in pathnames]

vehicles = load_images('data/vehicles/*/*.png')
non_vehicles = load_images('data/non-vehicles/*/*.png')

print('Loaded {} vehicle and {} non-vehicle images'.format(len(vehicles), len(non_vehicles)))

Loaded 8792 vehicle and 8968 non-vehicle images


## Extract features

In [2]:
import cv2
import numpy as np
from skimage.feature import hog

def extract_features(image, hog_orientations, hog_px_per_cell, hog_cells_per_block):
    """Extracts an unnormalized feature vectors from provided image"""
    
    grayscale = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    
    hog_features = hog(grayscale, hog_orientations, 
                       (hog_px_per_cell, hog_px_per_cell),
                       (hog_cells_per_block, hog_cells_per_block))
    
    return hog_features

# Feature extraction parameters
HOG_ORIENTATIONS = 9
HOG_PX_PER_CELL = 16
HOG_CELLS_PER_BLOCK = 2

vehicle_features = np.vstack([
    extract_features(image, HOG_ORIENTATIONS, HOG_PX_PER_CELL, HOG_CELLS_PER_BLOCK)
    for image in vehicles
])

non_vehicle_features = np.vstack([
    extract_features(image, HOG_ORIENTATIONS, HOG_PX_PER_CELL, HOG_CELLS_PER_BLOCK)
    for image in non_vehicles
])

print('Number of features: {}'.format(vehicle_features.shape[1]))

/home/goldob/miniconda3/envs/carnd-term1/lib/python3.5/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


Number of features: 324


## Split data into training & test sets

In [3]:
from sklearn.model_selection import train_test_split

# Use constant seed so that the train-test split remains the same between consecutive executions
SEED = 533343803

X = np.vstack([vehicle_features, non_vehicle_features])
y = np.hstack([np.ones(len(vehicles)), np.zeros(len(non_vehicles))])

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=SEED)

## Normalize features

In [4]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

## Train the classifier

In [ ]:
from sklearn.svm import SVC

clf = SVC(kernel='linear')
clf.fit(X_train, y_train)

print('Training accuracy: {:.3%}'.format(clf.score(X_train, y_train)))
print('Test accuracy: {:.3%}'.format(clf.score(X_test, y_test)))

# Vehicle detection

Being able to correctly classify images, we can now search for cars on a larger scene.

## Load reference image

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# Divide by 255 to normalize pixel values
image = mpimg.imread('test_images/test1.jpg') / 255

plt.imshow(image)
plt.show()

## Run a sliding window through the scene

In [ ]:
def draw_boxes(img, bboxes, color=(0, 0, 255), thick=6):
    """Draw bounding boxes for all window positions"""
    
    # Make a copy of the image
    imcopy = np.copy(img)
    # Iterate through the bounding boxes
    for bbox in bboxes:
        # Draw a rectangle given bbox coordinates
        cv2.rectangle(imcopy, bbox[0], bbox[1], color, thick)
    # Return the image copy with boxes drawn
    return imcopy
    
def slide_window(img, x_start_stop=[None, None], y_start_stop=[None, None], 
                    xy_window=(64, 64), xy_overlap=(0.5, 0.5)):
    """Compute positions of a sliding window with given parameters"""
    
    # If x and/or y start/stop positions not defined, set to image size
    if x_start_stop[0] == None: x_start_stop[0] = 0
    if y_start_stop[0] == None: y_start_stop[0] = 0
    if x_start_stop[1] == None: x_start_stop[1] = img.shape[1]
    if y_start_stop[1] == None: y_start_stop[1] = img.shape[0]
    
    # Compute the span of the region to be searched
    x_span = x_start_stop[1] - x_start_stop[0]
    y_span = y_start_stop[1] - y_start_stop[0]
    
    # Compute the number of pixels per step in x/y
    x_pix_per_step = int(xy_window[0] * (1 - xy_overlap[0]))
    y_pix_per_step = int(xy_window[1] * (1 - xy_overlap[1]))
    
    # Compute the number of windows in x/y
    x_windows = int(1 + ((x_span - xy_window[0]) / x_pix_per_step))
    y_windows = int(1 + ((y_span - xy_window[1]) / y_pix_per_step))
    
    # Initialize a list to append window positions to
    window_list = []
    # Loop through finding x and y window positions
    for ny in range(y_windows):
        for nx in range(x_windows):

            # Calculate each window position
            x1 = x_start_stop[0] + nx*x_pix_per_step
            y1 = y_start_stop[0] + ny*y_pix_per_step
            
            x2 = x1 + xy_window[0]
            y2 = y1 + xy_window[1]
            
            # Append window position to list
            window_list.append(((x1, y1), (x2, y2)))
            
    # Return the list of windows
    return window_list

bboxes = slide_window(image)
bboxes_image = draw_boxes(image, bboxes)

plt.imshow(bboxes_image)
plt.show()